In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns', 500)

In [2]:
filename = 'cleaned_fraudTrain.csv'

In [3]:
# import Dataset
path = '../dataset/midterm_dataset/'+filename
df = pd.read_csv(path)

row_num = df.shape[0]
feature_num = df.shape[1]

print(df.shape)
df.head()

(1296675, 14)


,category,amt,is_male,lat,long,city_pop,job,unix_time,is_fraud,state_city,age,trans_time,trans_year,trans_month
0,misc_net,4.97,0,36.0788,-81.1781,3495,"Psychologist, counselling",1325376018,0,"28654, Moravian Falls, NC",31,Late Night,2019,1
1,grocery_pos,107.23,0,48.8878,-118.2105,149,Special educational needs teacher,1325376044,0,"99160, Orient, WA",41,Late Night,2019,1
2,entertainment,220.11,1,42.1808,-112.2620,4154,Nature conservation officer,1325376051,0,"83252, Malad City, ID",57,Late Night,2019,1
3,gas_transport,45.00,1,46.2306,-112.1138,1939,Patent attorney,1325376076,0,"59632, Boulder, MT",52,Late Night,2019,1
4,misc_pos,41.96,1,38.4207,-79.4629,99,Dance movement psychotherapist,1325376186,0,"24433, Doe Hill, VA",33,Late Night,2019,1


In [4]:
label = 'is_fraud'
label

'is_fraud'

In [5]:
features = df.loc[:, df.columns != label].columns.tolist()
print(features)

['category', 'amt', 'is_male', 'lat', 'long', 'city_pop', 'job', 'unix_time', 'state_city', 'age', 'trans_time', 'trans_year', 'trans_month']


<h1> Decision Tree

<h2> Intial Entropy 'is_fraud'

In [532]:
#find intial Entropy 'is_fraud'
print(df[label].value_counts())
p_1_fraud = df[label].value_counts()[1]/len(df[label])
print('Probability Yes :', p_1_fraud)
p_0_fraud = df[label].value_counts()[0]/len(df[label])
print('Probability No :', p_0_fraud)

0    1289169
1       7506
Name: is_fraud, dtype: int64
Probability Yes : 0.005788651743883394
Probability No : 0.9942113482561166


In [533]:
intial_entropy = -p_1_fraud*np.log2(p_1_fraud)-p_0_fraud*np.log2(p_0_fraud)
intial_entropy

0.05135152470435041

In [6]:
def find_prob_each(label, feature, df=df):
    """
    -helper function for group_entropy function
    -and also give probability often its occures compare to the label
    
    return dict of probability of feature
    """

    temp_df = df.groupby([feature, label])
    prob_list = []
    if len(temp_df.size())< 2000 and feature not in ['age', 'lat', 'long', 'city_pop']:  
        for r in range(len(temp_df.size())):
                feature_prob = temp_df.size().values[r]/ \
                    len(df[feature][df[feature] == temp_df.size().index[r][0]])
                prob_feature = {}
                prob_feature[temp_df.size().index[r]] = feature_prob
                featu = ''
                if len(prob_list) == 0:
                    featu = temp_df.size().index[r][0]
                    prob_list.append({featu : [0.0, 0.0]})
                    if temp_df.size().index[r][1] == 1:
                        prob_list[-1][featu][1] = feature_prob
                    else:
                        prob_list[-1][featu][0] = feature_prob
                elif temp_df.size().index[r][0] in prob_list[-1]: #append 2nd value
#                     print(df.groupby([feature, label]).size().index[r][0], r,feature_prob, temp_df.size().index[r][1])
#                     print(prob_list[-1][temp_df.size().index[r][0]])
                    if temp_df.size().index[r][1] == 1:
                        prob_list[-1][temp_df.size().index[r][0]][1] = feature_prob
                    else:
                        prob_list[-1][temp_df.size().index[r][0]][0] = feature_prob
                else:
                    featu = temp_df.size().index[r][0]
                    prob_list.append({featu : [0.0, 0.0]})
                    if temp_df.size().index[r][1] == 1:
                        prob_list[-1][featu][1] = feature_prob
                    else:
                        prob_list[-1][featu][0] = feature_prob
                    
    else: #numeric feature
        greater_than_eq_df = df[[label, feature]][df[feature] >=df[feature].mean()]
        less_than_df = df[[label, feature]][df[feature] < df[feature].mean()]
        condition_dfs = [greater_than_eq_df, less_than_df]
    
        prob_list = [{'greater_than_eq_mean': []}, {'less_than_mean': []}]
    
        for d in condition_dfs:
            for r in range(len(d[label].value_counts())):
                feature_prob = d[label].value_counts()[r]/len(d)
#                 print(feature_prob)
                if  d.equals(greater_than_eq_df):
                    prob_list[0]['greater_than_eq_mean'].append(feature_prob)

                else:
                    prob_list[1]['less_than_mean'].append(feature_prob)
    p_dict = {}
    [p_dict.update(i) for i in prob_list]
    
    return p_dict

In [536]:
import math
def group_entropy(feature, df=df):
    """
    give most pure feature in sorted_feature part
    note: the most pure value = 0
    """
    an_entropy = {}
    prob_dict = find_prob_each(label, feature, df)
    
    for key in prob_dict:
        index0 = prob_dict[key][0]*np.log2(prob_dict[key][0])
        index1 = prob_dict[key][1]*np.log2(prob_dict[key][1])
        if math.isnan(index0): index0 = 0
        if math.isnan(index1): index1 = 0
        an_entropy[key] = -index0 -index1

    output = {
        feature: an_entropy,
        'sorted_feature' : []
    }
    t = [(k, v) for k, v in output[feature].items()]
    t.sort(key=lambda x : x[1])
    output['sorted_feature'] = t

    return output

In [538]:
def info_gain(intial_entropy, feature, df=df):
    """
    float, float, list
    find information gain
    
    output: float
    """
    g_entropy = group_entropy(feature, df)
    total_entropy = 0
    length_df = len(df)
    
    for i in g_entropy[feature]:
        if math.isnan((len(df[feature][df[feature] == i])/length_df) * g_entropy[feature][i]):
            total_entropy += 0
        else:
            total_entropy += (len(df[feature][df[feature] == i])/length_df) * g_entropy[feature][i]
#         print(i)
    return intial_entropy - (total_entropy)

In [540]:
import time
start_time = time.time()
df_ig = {'information_gain': []}

for feature in features:
    df_ig['information_gain'].append(info_gain(intial_entropy, feature))
    print(feature)
df_ig = pd.DataFrame.from_dict(df_ig)
df_ig.index = features
print("--- %s seconds ---" % (time.time() - start_time))
df_ig

category
amt
is_male
lat
long
city_pop


<ipython-input-536-32d36d306264>:11: RuntimeWarning: divide by zero encountered in log2
  index0 = prob_dict[key][0]*np.log2(prob_dict[key][0])
<ipython-input-536-32d36d306264>:11: RuntimeWarning: invalid value encountered in double_scalars
  index0 = prob_dict[key][0]*np.log2(prob_dict[key][0])
<ipython-input-536-32d36d306264>:12: RuntimeWarning: divide by zero encountered in log2
  index1 = prob_dict[key][1]*np.log2(prob_dict[key][1])
<ipython-input-536-32d36d306264>:12: RuntimeWarning: invalid value encountered in double_scalars
  index1 = prob_dict[key][1]*np.log2(prob_dict[key][1])


job
unix_time


<ipython-input-536-32d36d306264>:11: RuntimeWarning: divide by zero encountered in log2
  index0 = prob_dict[key][0]*np.log2(prob_dict[key][0])
<ipython-input-536-32d36d306264>:11: RuntimeWarning: invalid value encountered in double_scalars
  index0 = prob_dict[key][0]*np.log2(prob_dict[key][0])
<ipython-input-536-32d36d306264>:12: RuntimeWarning: divide by zero encountered in log2
  index1 = prob_dict[key][1]*np.log2(prob_dict[key][1])
<ipython-input-536-32d36d306264>:12: RuntimeWarning: invalid value encountered in double_scalars
  index1 = prob_dict[key][1]*np.log2(prob_dict[key][1])


state_city
age
trans_time
trans_year
trans_month
--- 441.23811388015747 seconds ---


,information_gain
category,0.003107
amt,0.051352
is_male,0.000042
lat,0.051352
long,0.051352
city_pop,0.051352
job,0.003134
unix_time,0.051352
state_city,0.007758
age,0.051352


<h2> Information Gain table

In [543]:
df_ig.sort_values(by=['information_gain'], inplace=True, ascending=False)
df_ig

,information_gain
amt,0.051352
lat,0.051352
long,0.051352
city_pop,0.051352
unix_time,0.051352
age,0.051352
state_city,0.007758
trans_time,0.005617
job,0.003134
category,0.003107


In [544]:
df_ig.to_csv('info_gain_all_feature.csv')

In [7]:
df_ig = pd.read_csv('info_gain_all_feature.csv')
df_ig = df_ig.rename(columns={'Unnamed: 0' : 'features'})
df_ig.set_index("features", inplace = True)
df_ig

,information_gain
features,
amt,0.051352
lat,0.051352
long,0.051352
city_pop,0.051352
unix_time,0.051352
age,0.051352
state_city,0.007758
trans_time,0.005617
job,0.003134


<h2> Training

In [8]:
X = df[features]

In [9]:
y = df[label]

In [10]:
#containing dict to decrease workload of model
store_prob_dict = {}

In [11]:
def extract_feature_point(feature, value, df=df, store_prob_dict=store_prob_dict):
    #check type
    if type(value) != str and feature not in ['is_male', 'trans_year', 'trans_month']:
        mean_val = df[feature].mean()
        if value >= mean_val:
            value = 'greater_than_eq_mean'
        else:
            value = 'less_than_mean'
    
    print('value', value)
        
    if feature not in store_prob_dict:
        prob_dict = find_prob_each(label, feature, df)
        store_prob_dict.update({feature : prob_dict})
        print(feature)
        print('store_dict length:', len(store_prob_dict))
        
    
    #append value
    if value not in store_prob_dict[feature]:
        #return 0 when hv no data
        return [0, 0]
    elif value in store_prob_dict[feature]:
        #return point
        return [
            100*store_prob_dict[feature][value][0]*float(df_ig.loc[feature,:]),
            100*store_prob_dict[feature][value][1]*float(df_ig.loc[feature,:])
        ]

In [12]:
def row_desicion_tree_predictor(an_row=[], features=[], df=df, store_prob_dict=store_prob_dict):
    not_fraud = []
    fraud = []
#     an_row = an_df_row.values.tolist()[0]
    for i, feature in enumerate(features):
        point = extract_feature_point(feature, an_row[i], df, store_prob_dict)
        not_fraud.append(point[0])
        fraud.append(point[1])
#         print(feature,':',point,'done')
    print('predicting..')
#     print(1)
    #predicting
    nfra_mean = np.mean(not_fraud)
    fra_mean = np.mean(fraud)
    if nfra_mean >= fra_mean:
        return 0
    else:
        return 1

In [13]:
def desicion_tree_predictor(X, features):
    """
    return list of predicting point
    """
    x_list = X.values.tolist()
    return [row_desicion_tree_predictor(row, features) for row in x_list]

In [ ]:
start_time = time.time()
predicted_list = desicion_tree_predictor(X, features)
print("--- %s seconds ---" % (time.time() - start_time))
predicted_list

In [ ]:
predicted_list

<H2> Import test data

In [14]:
# import Dataset
path = '../dataset/midterm_dataset/cleaned_fraudTest.csv'
df_test = pd.read_csv(path)

row_num = df_test.shape[0]
feature_num = df_test.shape[1]

print(df_test.shape)
df_test.head()

(555719, 14)


,category,amt,is_male,lat,long,city_pop,job,unix_time,is_fraud,state_city,age,trans_time,trans_year,trans_month
0,personal_care,2.86,1,33.9659,-80.9355,333497,Mechanical engineer,1371816865,0,"29209, Columbia, SC",52,Morning,2020,6
1,personal_care,29.84,0,40.3207,-110.4360,302,"Sales professional, IT",1371816873,0,"84002, Altonah, UT",30,Morning,2020,6
2,health_fitness,41.28,0,40.6729,-73.5365,34496,"Librarian, public",1371816893,0,"11710, Bellmore, NY",50,Morning,2020,6
3,misc_pos,60.05,1,28.5697,-80.8191,54767,Set designer,1371816915,0,"32780, Titusville, FL",33,Morning,2020,6
4,travel,3.19,1,44.2529,-85.0170,1126,Furniture designer,1371816917,0,"49632, Falmouth, MI",65,Morning,2020,6


In [16]:
X_test = df_test[features]
X_test.head()

,category,amt,is_male,lat,long,city_pop,job,unix_time,state_city,age,trans_time,trans_year,trans_month
0,personal_care,2.86,1,33.9659,-80.9355,333497,Mechanical engineer,1371816865,"29209, Columbia, SC",52,Morning,2020,6
1,personal_care,29.84,0,40.3207,-110.4360,302,"Sales professional, IT",1371816873,"84002, Altonah, UT",30,Morning,2020,6
2,health_fitness,41.28,0,40.6729,-73.5365,34496,"Librarian, public",1371816893,"11710, Bellmore, NY",50,Morning,2020,6
3,misc_pos,60.05,1,28.5697,-80.8191,54767,Set designer,1371816915,"32780, Titusville, FL",33,Morning,2020,6
4,travel,3.19,1,44.2529,-85.0170,1126,Furniture designer,1371816917,"49632, Falmouth, MI",65,Morning,2020,6


In [17]:
y_test = df_test[label]
y_test.head()

0    0
1    0
2    0
3    0
4    0
Name: is_fraud, dtype: int64

<h2> Prediction

In [19]:
import time
start_time = time.time()
predicted_test_list = desicion_tree_predictor(X_test, features)
print("--- %s seconds ---" % (time.time() - start_time))
predicted_test_list

eater_than_eq_mean
value Firefighter
value greater_than_eq_mean
value 90650, Norwalk, CA
value greater_than_eq_mean
value Night
value 2020
value 12
predicting..
value entertainment
value less_than_mean
value 0
value less_than_mean
value less_than_mean
value less_than_mean
value Occupational psychologist
value greater_than_eq_mean
value 74536, Clayton, OK
value greater_than_eq_mean
value Night
value 2020
value 12
predicting..
value kids_pets
value less_than_mean
value 1
value greater_than_eq_mean
value greater_than_eq_mean
value less_than_mean
value Community arts worker
value greater_than_eq_mean
value 46322, Highland, IN
value less_than_mean
value Night
value 2020
value 12
predicting..
value shopping_net
value greater_than_eq_mean
value 0
value less_than_mean
value less_than_mean
value less_than_mean
value Electrical engineer
value greater_than_eq_mean
value 72042, De Witt, AR
value less_than_mean
value Night
value 2020
value 12
predicting..
value misc_net
value less_than_mean
value 0

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [1]:
y_pred = predicted_test_list
len(y_pred)

NameError: name 'predicted_test_list' is not defined

In [32]:
len(list(y_test))

555719

In [45]:
len(list(y_test))-2145

553574

In [42]:
comp_confmat(y_test, predicted_test_list)[0][0]

553574.0

<h2> Evaluation

In [1]:
def comp_confmat(actual, predicted):

    classes = np.unique(actual) # extract the different classes
    matrix = np.zeros((len(classes), len(classes))) # initialize the confusion matrix with zeros

    for i in range(len(classes)):
        for j in range(len(classes)):

            matrix[i, j] = np.sum((actual == classes[i]) & (predicted == classes[j]))

    return matrix

In [2]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

In [3]:
def comp_f1_score(true_positive, false_positive, false_negative):
    return 2*true_positive/(2*true_positive+ false_positive+ false_negative)


In [4]:
def comp_precision(true_positive, false_positive):
    return true_positive/(true_positive+false_positive)

In [5]:
def comp_recall(true_positive, false_negative):
    return true_positive/(true_positive+false_negative)

<h2>The result:

In [48]:
len(y_test), len(y_pred)

(555719, 555719)

<h3>Confustion Matrix

In [33]:
comp_confmat(y_test, predicted_test_list)

array([[553574.,      0.],
       [  2145.,      0.]])

<h3> Accuracy percentage

In [43]:
accuracy_metric(y_test, predicted_test_list)

99.61401355721146

From Confustion Matrix
<h3> Precision

In [7]:
comp_precision(553574,0)

1.0

From Confustion Matrix
<h3> Recall

In [8]:
comp_recall(553574, 2145)

0.9961401355721147

From Confustion Matrix
<h3> F1-Score

In [10]:
comp_f1_score(553574, 0, 2145)

0.9980663359455076